## <HR>Creation of Environmental Features for Wildfires Incidents Using NCEP data<HR>

#### Import the Required Packages

In [1]:
import os
import sys
import json
import glob
import time
import pickle
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import concurrent.futures
import dask.dataframe as dd
from tqdm.notebook import tqdm

In [2]:
warnings.filterwarnings('ignore')

In [3]:
DATA_PATH = "data/"

#### 'EnvironmentalFeaturesGenerator' class to extract the environmental features

In [4]:
class EnvironmentalFeaturesGenerator:
    """FeaturesGenerator to generate environmental features.
    
    It generates the features like Air_Temperature('air'), Geopotential_Height('hgt'), 
    Relative_Humidity('rhum'), East_Wind/Zonal_Wind('uwnd'), North_Wind/Meridional_Wind('vwnd'),  
    Tropopause_Pressure('trpp'), Tropopause_Temperature('trpt'), Surface_Potential_Temperature('srfpt') at different pressure level using NCEP data. 
    
    NCEP data link:
    https://psl.noaa.gov/data/gridded/data.ncep.html
    https://www.ncei.noaa.gov/erddap/convert/oceanicAtmosphericVariableNames.htmlTable
    
    """
    
    def __init__(self, wildfires_data, ncep_data, ncep_sfc_data):
        self.wildfires_data = wildfires_data
        self.ncep_data = ncep_data
        self.levels = [100, 150, 200, 500, 1000]
        self.ncep_data_vars = set(ncep_data[list(ncep_data)[0]]) - set(['head'])
        self.ncep_sfc_data = ncep_sfc_data
        self.ncep_sfc_data_vars = set(ncep_sfc_data[list(ncep_sfc_data)[0]]) - set(['head'])
        
    def _extract_features(self, row):
        """extract the environmental features for particular wildfire incident"""
        ncep_data = self.ncep_data
        ncep_sfc_data = self.ncep_sfc_data
        date = row['date']
        features = dict(row)
        #reduce the dimensions of ncep_data(xarray dataset) by fixing coordinates(lon,lat)
        #and then convert it to dataframe
        ncep_data = ncep_data[date.year] \
                            .sel(lon=row['longitude'], lat=row['latitude'], method='nearest') \
                            .to_dask_dataframe() \
                            .compute() \
                            .set_index(['level','time'])
        #reduce the dimensions of ncep_sfc_data(xarray dataset) by fixing coordinates(lon,lat)
        #and then convert it to dataframe
        ncep_sfc_data = ncep_sfc_data[date.year] \
                            .sel(lon=row['longitude'], lat=row['latitude'], method='nearest') \
                            .to_dask_dataframe() \
                            .compute() \
                            .set_index(['time'])

        for level in self.levels:
            #features at different pressure level
            point = ncep_data.loc[level]
            p1w = point.rolling(7).mean()  # 1 Week mean
            p2w = point.rolling(14).mean() # 2 Week mean
            p3w = point.rolling(21).mean() # 3 Week mean
            # 
            v0w = point.loc[date]
            v1w = p1w.loc[date]
            v2w = p2w.loc[date]
            v3w = p3w.loc[date]
            #
            for data_var in self.ncep_data_vars:
                features["{0}_0w_lvl_{1}".format(data_var,level)] = v0w[data_var]
                features["{0}_1w_lvl_{1}".format(data_var,level)] = v1w[data_var]
                features["{0}_2w_lvl_{1}".format(data_var,level)] = v2w[data_var]
                features["{0}_3w_lvl_{1}".format(data_var,level)] = v3w[data_var]
        #features at surface level
        point = ncep_sfc_data
        p1w = point.rolling(7).mean()  # 1 Week mean
        p2w = point.rolling(14).mean() # 2 Week mean
        p3w = point.rolling(21).mean() # 3 Week mean
        # 
        v0w = point.loc[date]
        v1w = p1w.loc[date]
        v2w = p2w.loc[date]
        v3w = p3w.loc[date]
        #
        for data_var in self.ncep_sfc_data_vars:
            features["{0}_0w".format(data_var)] = v0w[data_var]
            features["{0}_1w".format(data_var)] = v1w[data_var]
            features["{0}_2w".format(data_var)] = v2w[data_var]
            features["{0}_3w".format(data_var)] = v3w[data_var] 

        return features
    
    def _get_day_info(self, df_features):
        """ """
        df_features['day'] = df_features.date.dt.day
        df_features['month'] = df_features.date.dt.month
        df_features['year']= df_features.date.dt.year
        df_features['week_day'] = df_features.date.dt.weekday
        df_features['weekofyear'] = df_features.date.dt.weekofyear
        df_features['is_winter'] = df_features.apply(lambda x: int(x['month'] in [12,1,2]), axis=1)
        df_features['is_autumn'] = df_features.apply(lambda x: int(x['month'] in [9,10,11]), axis=1)
        df_features['is_summer'] = df_features.apply(lambda x: int(x['month'] in [6,7,8]), axis=1)
        df_features['is_spring'] = df_features.apply(lambda x: int(x['month'] in [3,4,5]), axis=1)
        return df_features
        
    def _thread_func(self, year):
        wildfires_data = self.wildfires_data
        file_path = DATA_PATH + 'features/' + str(year) + '.pickle'
        df_features = {}
        try:
            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f:
                    df_features = pickle.load(f)
                    print('Loaded: ',len(df_features)) 
            start_year, end_year = year, year+1
            df_subsample = wildfires_data.query('(date >= @start_year) & (date < @end_year)')
            for row_id, row in tqdm(df_subsample.iterrows(), total=df_subsample.shape[0], desc="Year "+str(year)):
                if row_id not in df_features:
                    df_features[row_id] = self._extract_features(row)
                    with open(file_path, 'wb') as f:
                        pickle.dump(df_features, f)
            df_features = self._get_day_info(pd.DataFrame(df_features.values()))           
            df_features.set_index('fire_id').to_csv(DATA_PATH + 'features/' + str(year) + '.csv')
        except Exception as e:
            print(e)
            
    def start(self, years):
        """start extracting the features using multiple threads, one thread per year """
        print("="*5,"Features Extraction has started","="*5)
        time.sleep(0.5)
        start = time.time()
        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            for year in years:
                executor.submit(self._thread_func, year)
        end = time.time()
        print("\n","="*5,"Features Extraction has finished","="*5)
        print("Total time taken", end-start, " sec")

#### 'load_Dataset' method to load the data into the memory

In [5]:
def load_Dataset(years):
    """load the wildfire data and NCEP data into the memory"""
    ncep_data, ncep_sfc_data = dict(), dict()
    #Mutli-dimensional ncep data at different pressure level and surface level
    for year in years:
        ncep_data[year] = xr.open_mfdataset('data/ncep/*.'+str(year)+'.nc', combine='by_coords', parallel=True)
        ncep_sfc_data[year] = xr.open_mfdataset('data/ncep/ncep_sfc/*.'+str(year)+'.nc', combine='by_coords', parallel=True)
    #dataset for wildfire incidents
    wildfires_train = pd.read_csv('data/wildfires_train.csv', parse_dates=['date'])
    wildfires_train['fire_type_name_en'] = \
        wildfires_train.fire_type_name.map(
        json.load(open('data/fire_type_name_en.json','r',encoding='utf-8')),
        na_action='ignore')
    return wildfires_train, ncep_data, ncep_sfc_data

#### Extract Environmental Features

In [6]:
years = range(2012,2016)
generator = EnvironmentalFeaturesGenerator(*load_Dataset(years))
generator.start(years)

===== Features Extraction has started =====
Loaded:  9593



Loaded:  16418



Loaded:  28179



Loaded:  35530




 ===== Features Extraction has finished =====
Total time taken 98.7444679737091  sec


In [7]:
years = range(2016,2020)
generator = EnvironmentalFeaturesGenerator(*load_Dataset(years))
generator.start(years)

===== Features Extraction has started =====
Loaded:  16011



Loaded:  23070


Loaded:  22820


Loaded:  23250






 ===== Features Extraction has finished =====
Total time taken 91.26349234580994  sec


#### Merge all years features data

In [8]:
# pickle_env_features = {}
# for file in glob.glob(DATA_PATH + 'features/'+'*.pickle'):
#     with open(file, 'rb') as f:
#         pickle_env_features.update(pickle.load(f))  
# with open(DATA_PATH + 'env_features.pickle', 'wb') as f:
#     pickle.dump(pickle_env_featuress, f)

In [9]:
df_env_features = dd.read_csv(DATA_PATH + 'features/*.csv', parse_dates=['date']).compute()
df_env_features.sort_values("fire_id", inplace = True)

In [10]:
df_env_features.shape

(174871, 188)

#### Missing Values in features data

In [35]:
missing_value = pd.DataFrame(df_env_features.isnull().sum(axis=0), columns=['Count'])
missing_value['Pert'] = missing_value.Count*100/df_env_features.shape[0]
missing_value.sort_values(by='Count', ascending=False, inplace=True)
missing_value[missing_value.Count>0]

,Count,Pert
srfpt_3w_lvl_150,59391,33.962750
srfpt_3w_lvl_200,59391,33.962750
srfpt_3w_lvl_100,59391,33.962750
srfpt_2w_lvl_200,59219,33.864391
srfpt_2w_lvl_100,59219,33.864391
...,...,...
srfpt_1w_lvl_1000,231,0.132097
rhum_1w,231,0.132097
rhum_1w_lvl_1000,231,0.132097
air_1w_lvl_200,231,0.132097


In [36]:
#Fill the missing values
df_env_features.fillna(0, inplace=True)

In [40]:
#save environmental features into csv file
df_env_features.to_csv(DATA_PATH + 'env_features.csv', index=False)